In [1]:
import pandas as pd
import numpy as np

place_df = pd.read_csv("관광지통합.csv", encoding="UTF-8", index_col=0)
place_df.head()

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
0,0,롯데백화점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,0.116515,을지로입구역 8번출구
1,1,신세계백화점/본점신관,서울 중구 소공로 63,37.560294,126.980503,쇼핑,0.055278,신세계면세점
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,0.055278,신세계면세점
3,3,두산타워,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,0.090793,동대문디자인플라자
4,4,현대시티아울렛/동대문점,서울 중구 장충단로13길 2,37.568071,127.006645,쇼핑,0.283583,동대문역사문화공원역 1번출구 뒤편


### 전처리 및 EDA

In [2]:
# 결측치 확인
missing= place_df.shape[0] - place_df.count()
print(missing)

순위         0
관광지명       0
주소         0
위도         0
경도         0
분류         0
최단거리       0
최단거리대여소    0
dtype: int64


In [3]:
# km 단위로 되어있는 최단거리 단위 m로 바꿔주기
place_df["최단거리"] = place_df["최단거리"].apply(lambda x : x * 1000)
place_df.head()

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
0,0,롯데백화점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
1,1,신세계백화점/본점신관,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
3,3,두산타워,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,90.793468,동대문디자인플라자
4,4,현대시티아울렛/동대문점,서울 중구 장충단로13길 2,37.568071,127.006645,쇼핑,283.582809,동대문역사문화공원역 1번출구 뒤편


In [4]:
# 가장 가까운 따릉이 정류소와의 거리 데이터 관련 정보 확인
# => 성인 평균 걷는 속도 고려했을 때 200 미터 이내에 따릉이 정류소가 있는 관광지만 선별하는 것이 좋다고 판단.
# => (평균 걷는속도 시속 4km인데 최소 5분 거리 이내에 있어야 한다고 판단. 단순 위도 경도로 측정하는 것과 실제 걷는 거리의 차이가 
# 있을 것이기에 4000 / 60 * 3 해서 나오는 값인 200m 이내로 결정)
place_df["최단거리"].describe()

# 근데 describe 함수 결과가 이상해서 확인해보니...

count    3.960000e+02
mean     3.745462e+04
std      7.415484e+05
min      3.716450e+00
25%      9.651978e+01
50%      1.519576e+02
75%      2.379107e+02
max      1.475682e+07
Name: 최단거리, dtype: float64

In [5]:
# 위도경도가 0으로 들어가서 최단거리가 매우 높게 측정된 값이 생김... 제거해주기
place_df[place_df["최단거리"] > 1000]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
160,60,살곶이체육공원,서울특별시 성동구 중랑천서자전거길 177,0.0,0.0,자연관광지,14756816.86,항동지구 7단지 정문


In [6]:
# 제거하고 다시 한 번 확인해보니 정상적으로 뜨는 것을 확인할 수 있음
place_df = place_df.drop(160)
place_df["최단거리"].describe()

count    395.000000
mean     190.414148
std      142.464283
min        3.716450
25%       96.519781
50%      151.667131
75%      236.064673
max      815.314570
Name: 최단거리, dtype: float64

In [7]:
# 따릉이 정류소와의 최단거리가 200미터 이내인 것들만 남겨주기
# => 총 데이터가 251개의 관광지로 줄어든 것을 확인할 수 있음
place_df = place_df[place_df["최단거리"] <= 200]
place_df["최단거리"].describe()

count    251.000000
mean     110.066385
std       48.990080
min        3.716450
25%       70.740648
50%      109.227988
75%      144.288271
max      198.419972
Name: 최단거리, dtype: float64

In [8]:
# 남은 251 개의 관광지 데이터의 관광지 "분류" 컬럼 값의 분포 확인해보기 => 추후 서비스에서 중요한 feature로 사용될 예정
place_df["분류"].value_counts()

쇼핑               133
문화시설              29
레포츠(육상/해상/항공)     20
문화관광              19
공연/행사             14
자연관광지             13
역사관광지              6
역사관광               5
기타관광               5
휴양관광지              4
기타관광지              3
Name: 분류, dtype: int64

In [9]:
# 분류 항목에서 어떤 항목은 너무 적고 쇼핑 같은 항목은 너무 많은 문제... 또 너무 세분화되어 있는 문제
# 관광지 데이터를 직접 확인해보고 분류를 합치기로 결정함.
# 자연관광지 => 휴양관광지로 편입
# 역사관광 => 역사관광지로 편입
# 문화시설 => 문화관광으로 편입
# 기타관광/기타관광지 삭제
# 공연/행사에서 영화관 관광지는 삭제 후 문화관광으로 편입
place_df["분류"].replace({"자연관광지":"휴양관광지", "역사관광":"역사관광지", "문화시설":"문화관광"}, inplace=True)
place_df["분류"].value_counts()

쇼핑               133
문화관광              48
레포츠(육상/해상/항공)     20
휴양관광지             17
공연/행사             14
역사관광지             11
기타관광               5
기타관광지              3
Name: 분류, dtype: int64

In [10]:
place_df[place_df["분류"] == "기타관광"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
214,18,청와대,서울 종로구 청와대로 1,37.582369,126.980198,기타관광,140.146442,청와대앞길
224,28,청와대/정문,서울 종로구 청와대로 1,37.582369,126.980198,기타관광,140.146442,청와대앞길
257,61,청와대/춘추문,서울 종로구 청와대로 1,37.582369,126.980198,기타관광,140.146442,청와대앞길
276,80,청와대/춘추관,서울 종로구 청와대로 1,37.582369,126.980198,기타관광,140.146442,청와대앞길
280,84,청와대/영빈문,서울 종로구 청와대로 1,37.582369,126.980198,기타관광,140.146442,청와대앞길


In [11]:
place_df = place_df.drop(index=[214, 224, 257, 276, 280])
place_df["분류"].value_counts()

쇼핑               133
문화관광              48
레포츠(육상/해상/항공)     20
휴양관광지             17
공연/행사             14
역사관광지             11
기타관광지              3
Name: 분류, dtype: int64

In [12]:
place_df[place_df["분류"] == "기타관광지"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
38,38,장충체육관,서울 중구 동호로 241,37.558170,127.006795,기타관광지,45.59097,장충체육관
75,75,남산팔각정,서울 중구,37.556028,126.972224,기타관광지,125.32757,서울역9번출구
93,93,서울정원박람회/(백범광장),서울 중구,37.556028,126.972224,기타관광지,125.32757,서울역9번출구


In [13]:
place_df = place_df.drop(index=[38, 75, 93])
place_df["분류"].value_counts()

쇼핑               133
문화관광              48
레포츠(육상/해상/항공)     20
휴양관광지             17
공연/행사             14
역사관광지             11
Name: 분류, dtype: int64

In [14]:
# 이 중에서 영화관은 제거
place_df[place_df["분류"] == "공연/행사"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
15,15,국립극장,서울 중구 장충단로 59,37.568043,127.009194,공연/행사,42.114914,동대문디자인플라자
24,24,충무아트센터,서울 중구 퇴계로 387,37.565690,127.014736,공연/행사,185.387691,신당역 12번 출구 뒤
45,45,대한극장,서울 중구 퇴계로 212,37.561159,126.995324,공연/행사,25.823902,충무로역 1번출구
65,65,메가박스/동대문,서울 중구 장충단로 247,37.566375,127.007386,공연/행사,122.595092,동대문역사문화공원역 1번출구 뒤편
80,80,국립정동극장,서울 중구 정동길 43,37.565865,126.972772,공연/행사,114.114119,시립미술관
110,10,CGV/왕십리,서울 성동구 왕십리광장로 17,37.561222,127.037136,공연/행사,40.033302,왕십리역 4번 출구 건너편
146,46,소월아트홀,서울 성동구 왕십리로 281,37.549734,127.044168,공연/행사,144.288271,서울숲 IT캐슬
221,25,CGV/피카디리1958,서울 종로구 돈화문로5가길 1,37.571936,126.991419,공연/행사,78.781210,종로3가역 8번출구
251,55,CGV/대학로,서울 종로구 대명길 28,37.583229,126.999989,공연/행사,125.757507,성균관대입구사거리
260,64,두산아트센터,서울 종로구 종로33길 15,37.572042,127.001509,공연/행사,96.290949,서울특별시 중부교육지원청


In [15]:
place_df = place_df.drop(index=[45, 65, 110, 221, 251, 278, 330, 372])
place_df["분류"].value_counts()

쇼핑               133
문화관광              48
레포츠(육상/해상/항공)     20
휴양관광지             17
역사관광지             11
공연/행사              6
Name: 분류, dtype: int64

In [16]:
# 5개 항목으로 정리 완료
place_df["분류"].replace({"공연/행사":"문화관광"}, inplace=True)
place_df["분류"].value_counts()

쇼핑               133
문화관광              54
레포츠(육상/해상/항공)     20
휴양관광지             17
역사관광지             11
Name: 분류, dtype: int64

In [17]:
# EDA를 진행하며 중복되어 있는 관광지데이터가 꽤 많음을 확인함.
# => 위치데이터가 일칲하는 데이터들 확인후 중복값 삭제 진행
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
12,12,롯데면세점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
15,15,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
31,31,청평화시장,서울 중구 청계천로 334,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
35,35,신평화패션타운,서울 중구 청계천로 298,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [18]:
# 생각보다 너무 많아서 하나씩 확인해보는 작업이 필요할 듯
# 지금부터 밑으로 쭈욱 노가다 진행...

place_df[place_df["위도"] == place_df.loc[2, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
1,1,신세계백화점/본점신관,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점


In [19]:
place_df = place_df.drop(index=[1])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
12,12,롯데면세점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
15,15,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
31,31,청평화시장,서울 중구 청계천로 334,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
35,35,신평화패션타운,서울 중구 청계천로 298,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
37,37,평화시장,서울 중구 청계천로 274,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [20]:
place_df[place_df["위도"] == place_df.loc[12, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
0,0,롯데백화점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
12,12,롯데면세점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구


In [21]:
place_df = place_df.drop(index=[12])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
15,15,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
31,31,청평화시장,서울 중구 청계천로 334,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
35,35,신평화패션타운,서울 중구 청계천로 298,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
37,37,평화시장,서울 중구 청계천로 274,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
39,39,황학동주방기구거리,서울 중구,37.556028,126.972224,쇼핑,125.327570,서울역9번출구
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [22]:
place_df[place_df["위도"] == place_df.loc[15, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
6,6,밀리오레/동대문점,서울 중구 장충단로 263,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
15,15,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
61,61,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
91,91,남평화상가,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자


In [23]:
place_df.loc[91, "관광지명"] = "남평화상가 + 밀리오레/동대문점"
place_df = place_df.drop(index=[6, 15,])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
31,31,청평화시장,서울 중구 청계천로 334,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
35,35,신평화패션타운,서울 중구 청계천로 298,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
37,37,평화시장,서울 중구 청계천로 274,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
39,39,황학동주방기구거리,서울 중구,37.556028,126.972224,쇼핑,125.327570,서울역9번출구
40,40,현대백화점면세점/동대문점,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,90.793468,동대문디자인플라자
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [24]:
place_df[place_df["위도"] == place_df.loc[31, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
5,5,이마트/청계천점,서울 중구 청계천로 4,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
31,31,청평화시장,서울 중구 청계천로 334,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
35,35,신평화패션타운,서울 중구 청계천로 298,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
37,37,평화시장,서울 중구 청계천로 274,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
41,41,롯데베네치아메가몰,서울 중구 청계천로 4,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
88,88,세운청계상가,서울 중구 청계천로 16,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬


In [25]:
# 여기서는 청계천 쇼핑 거리로 통합시키겠음.
place_df.loc[88, "관광지명"] = "청계천 쇼핑거리"
place_df = place_df.drop(index=[5, 31, 35, 37, 41])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
39,39,황학동주방기구거리,서울 중구,37.556028,126.972224,쇼핑,125.327570,서울역9번출구
40,40,현대백화점면세점/동대문점,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,90.793468,동대문디자인플라자
42,42,동대문APM상,서울 중구 마장로 4,37.568005,127.015818,쇼핑,108.075606,신당역 10번 출구
50,50,롯데백화점/에비뉴엘본점,서울 중구 남대문로 73,37.564214,126.981575,쇼핑,188.826124,을지로입구역 8번출구
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [26]:
place_df[place_df["위도"] == place_df.loc[39, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
9,9,남대문시장,서울 중구,37.556028,126.972224,쇼핑,125.32757,서울역9번출구
39,39,황학동주방기구거리,서울 중구,37.556028,126.972224,쇼핑,125.32757,서울역9번출구
60,60,만리재길,서울 중구,37.556028,126.972224,휴양관광지,125.32757,서울역9번출구
71,71,남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.32757,서울역9번출구
81,81,청계천공구상가,서울 중구,37.556028,126.972224,쇼핑,125.32757,서울역9번출구


In [27]:
# 여기서는 남대문시장으로 통합
place_df.loc[71, "관광지명"] = "만리재길+남산백범광장"
place_df = place_df.drop(index=[39, 60, 81])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
40,40,현대백화점면세점/동대문점,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,90.793468,동대문디자인플라자
42,42,동대문APM상,서울 중구 마장로 4,37.568005,127.015818,쇼핑,108.075606,신당역 10번 출구
50,50,롯데백화점/에비뉴엘본점,서울 중구 남대문로 73,37.564214,126.981575,쇼핑,188.826124,을지로입구역 8번출구
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
52,52,남산공원,서울 중구 소파로 46,37.556140,126.979130,문화관광,188.458164,도동삼거리
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [28]:
place_df[place_df["위도"] == place_df.loc[40, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
3,3,두산타워,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,90.793468,동대문디자인플라자
40,40,현대백화점면세점/동대문점,서울 중구 장충단로 275,37.568662,127.008718,쇼핑,90.793468,동대문디자인플라자


In [29]:
place_df = place_df.drop(index=[3])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
42,42,동대문APM상,서울 중구 마장로 4,37.568005,127.015818,쇼핑,108.075606,신당역 10번 출구
50,50,롯데백화점/에비뉴엘본점,서울 중구 남대문로 73,37.564214,126.981575,쇼핑,188.826124,을지로입구역 8번출구
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
52,52,남산공원,서울 중구 소파로 46,37.556140,126.979130,문화관광,188.458164,도동삼거리
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.564160,126.973808,문화관광,108.646323,시립미술관
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [30]:
place_df[place_df["위도"] == place_df.loc[42, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
19,19,ddp패션몰,서울 중구 마장로 22,37.568005,127.015818,쇼핑,108.075606,신당역 10번 출구
42,42,동대문APM상,서울 중구 마장로 4,37.568005,127.015818,쇼핑,108.075606,신당역 10번 출구


In [31]:
place_df = place_df.drop(index=[42])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
50,50,롯데백화점/에비뉴엘본점,서울 중구 남대문로 73,37.564214,126.981575,쇼핑,188.826124,을지로입구역 8번출구
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
52,52,남산공원,서울 중구 소파로 46,37.556140,126.979130,문화관광,188.458164,도동삼거리
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.564160,126.973808,문화관광,108.646323,시립미술관
54,54,롯데하이마트/청구점,서울 중구 다산로 21,37.567665,127.016085,쇼핑,77.372160,신당역 10번 출구
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [32]:
place_df[place_df["위도"] == place_df.loc[50, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
26,26,롯데백화점/에비뉴엘,서울 중구 남대문로 73,37.564214,126.981575,쇼핑,188.826124,을지로입구역 8번출구
50,50,롯데백화점/에비뉴엘본점,서울 중구 남대문로 73,37.564214,126.981575,쇼핑,188.826124,을지로입구역 8번출구


In [33]:
place_df = place_df.drop(index=[26])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
52,52,남산공원,서울 중구 소파로 46,37.556140,126.979130,문화관광,188.458164,도동삼거리
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.564160,126.973808,문화관광,108.646323,시립미술관
54,54,롯데하이마트/청구점,서울 중구 다산로 21,37.567665,127.016085,쇼핑,77.372160,신당역 10번 출구
58,58,대림상가,서울 중구 을지로 157,37.566120,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [34]:
place_df[place_df["위도"] == place_df.loc[51, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
21,21,덕수궁,서울 중구 세종대로 99,37.566331,126.9739,역사관광지,152.24812,시립미술관
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.9739,문화관광,152.24812,시립미술관
64,64,덕수궁,서울 중구 세종대로 99,37.566331,126.9739,역사관광지,152.24812,시립미술관


In [35]:
place_df = place_df.drop(index=[64])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
52,52,남산공원,서울 중구 소파로 46,37.556140,126.979130,문화관광,188.458164,도동삼거리
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.564160,126.973808,문화관광,108.646323,시립미술관
54,54,롯데하이마트/청구점,서울 중구 다산로 21,37.567665,127.016085,쇼핑,77.372160,신당역 10번 출구
58,58,대림상가,서울 중구 을지로 157,37.566120,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [36]:
place_df[place_df["위도"] == place_df.loc[52, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
14,14,남산공원,서울 중구 소파로 46,37.55614,126.97913,휴양관광지,188.458164,도동삼거리
52,52,남산공원,서울 중구 소파로 46,37.55614,126.97913,문화관광,188.458164,도동삼거리


In [37]:
place_df = place_df.drop(index=[52])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.564160,126.973808,문화관광,108.646323,시립미술관
54,54,롯데하이마트/청구점,서울 중구 다산로 21,37.567665,127.016085,쇼핑,77.372160,신당역 10번 출구
58,58,대림상가,서울 중구 을지로 157,37.566120,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [38]:
place_df[place_df["위도"] == place_df.loc[53, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
22,22,서울시립미술관,서울 중구 덕수궁길 61,37.56416,126.973808,문화관광,108.646323,시립미술관
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.56416,126.973808,문화관광,108.646323,시립미술관


In [39]:
place_df = place_df.drop(index=[22])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
54,54,롯데하이마트/청구점,서울 중구 다산로 21,37.567665,127.016085,쇼핑,77.372160,신당역 10번 출구
58,58,대림상가,서울 중구 을지로 157,37.566120,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [40]:
place_df[place_df["위도"] == place_df.loc[54, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
16,16,디오트쇼핑몰,서울 중구 다산로 293,37.567665,127.016085,쇼핑,77.37216,신당역 10번 출구
54,54,롯데하이마트/청구점,서울 중구 다산로 21,37.567665,127.016085,쇼핑,77.37216,신당역 10번 출구
95,95,GS더프레시/남산타운점,서울 중구 다산로 32,37.567665,127.016085,쇼핑,77.37216,신당역 10번 출구


In [41]:
place_df = place_df.drop(index=[54, 95])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
58,58,대림상가,서울 중구 을지로 157,37.566120,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [42]:
place_df[place_df["위도"] == place_df.loc[58, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
29,29,에이피엠플레이스,서울 중구 을지로 276,37.56612,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
58,58,대림상가,서울 중구 을지로 157,37.56612,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞
92,92,삼성전자서비스/을지로휴대폰센터,서울 중구 을지로 51,37.56612,127.007477,쇼핑,96.539864,동대문역사문화공원역 9번출구 앞


In [43]:
place_df = place_df.drop(index=[58, 92])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [44]:
place_df[place_df["위도"] == place_df.loc[68, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
13,13,롯데영플라자/명동점,서울 중구 남대문로 67,37.568713,126.982673,쇼핑,40.548823,광교사거리 남측
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측


In [45]:
place_df[place_df["위도"] == place_df.loc[71, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
9,9,남대문시장,서울 중구,37.556028,126.972224,쇼핑,125.32757,서울역9번출구
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.32757,서울역9번출구


In [46]:
place_df[place_df["위도"] == place_df.loc[73, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
61,61,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자


In [47]:
place_df[place_df["위도"] == place_df.loc[74, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
44,44,안중근의사기념관,서울 중구 소월로 91,37.55587,126.97687,문화관광,68.640827,도동삼거리
74,74,숭례문수입상가,서울 중구 소월로 3,37.55587,126.97687,쇼핑,68.640827,도동삼거리


In [48]:
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
78,78,충무아트센터,서울 중구 퇴계로 387,37.565690,127.014736,문화관광,185.387691,신당역 12번 출구 뒤
83,83,아트프라자,서울 중구 마장로1길 28,37.568446,127.012324,쇼핑,105.099813,ddp패션몰
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
97,97,서소문역사공원,서울 중구 칠패로 5,37.559443,126.968909,휴양관광지,58.204214,종로학원본원
98,98,동대문역사문화공원,서울 중구 을지로 281,37.567069,127.009899,문화관광,146.473176,동대문디자인플라자


In [49]:
place_df[place_df["위도"] == place_df.loc[78, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
24,24,충무아트센터,서울 중구 퇴계로 387,37.56569,127.014736,문화관광,185.387691,신당역 12번 출구 뒤
78,78,충무아트센터,서울 중구 퇴계로 387,37.56569,127.014736,문화관광,185.387691,신당역 12번 출구 뒤


In [50]:
place_df = place_df.drop(index=[24])
place_df[place_df.duplicated(["위도", "경도"])]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
...,...,...,...,...,...,...,...,...
342,46,압구정로데오거리,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.493480,127.061274,쇼핑,191.756747,대치역 7번출구


In [51]:
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
83,83,아트프라자,서울 중구 마장로1길 28,37.568446,127.012324,쇼핑,105.099813,ddp패션몰
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
97,97,서소문역사공원,서울 중구 칠패로 5,37.559443,126.968909,휴양관광지,58.204214,종로학원본원
98,98,동대문역사문화공원,서울 중구 을지로 281,37.567069,127.009899,문화관광,146.473176,동대문디자인플라자
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,휴양관광지,47.124576,이마트 버스정류소 옆


In [52]:
place_df[place_df["위도"] == place_df.loc[83, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
72,72,혜양엘리시움,서울 중구 마장로1길 18,37.568446,127.012324,쇼핑,105.099813,ddp패션몰
83,83,아트프라자,서울 중구 마장로1길 28,37.568446,127.012324,쇼핑,105.099813,ddp패션몰


In [53]:
place_df = place_df.drop(index=[72])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
97,97,서소문역사공원,서울 중구 칠패로 5,37.559443,126.968909,휴양관광지,58.204214,종로학원본원
98,98,동대문역사문화공원,서울 중구 을지로 281,37.567069,127.009899,문화관광,146.473176,동대문디자인플라자
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,휴양관광지,47.124576,이마트 버스정류소 옆
106,6,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,문화관광,47.124576,이마트 버스정류소 옆


In [54]:
place_df[place_df["위도"] == place_df.loc[91, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
61,61,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자


In [55]:
place_df[place_df["위도"] == place_df.loc[97, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
63,63,서소문성지역사박물관,서울 중구 칠패로 5,37.559443,126.968909,문화관광,58.204214,종로학원본원
97,97,서소문역사공원,서울 중구 칠패로 5,37.559443,126.968909,휴양관광지,58.204214,종로학원본원


In [56]:
place_df.loc[63, "분류"] = "역사관광지"
place_df = place_df.drop(index=[97])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
98,98,동대문역사문화공원,서울 중구 을지로 281,37.567069,127.009899,문화관광,146.473176,동대문디자인플라자
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,휴양관광지,47.124576,이마트 버스정류소 옆
106,6,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,문화관광,47.124576,이마트 버스정류소 옆
108,8,엔터식스/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편


In [57]:
place_df[place_df["위도"] == place_df.loc[98, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
43,43,동대문역사문화공원,서울 중구 을지로 281,37.567069,127.009899,휴양관광지,146.473176,동대문디자인플라자
98,98,동대문역사문화공원,서울 중구 을지로 281,37.567069,127.009899,문화관광,146.473176,동대문디자인플라자


In [58]:
place_df = place_df.drop(index=[43])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,휴양관광지,47.124576,이마트 버스정류소 옆
106,6,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,문화관광,47.124576,이마트 버스정류소 옆
108,8,엔터식스/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
116,16,센트라스상가/L동,서울 성동구 왕십리로 41,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬


In [59]:
place_df[place_df["위도"] == place_df.loc[101, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
100,0,이마트/성수점,서울 성동구 뚝섬로 379,37.539411,127.05294,쇼핑,47.124576,이마트 버스정류소 옆
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.05294,휴양관광지,47.124576,이마트 버스정류소 옆
106,6,서울숲,서울 성동구 뚝섬로 273,37.539411,127.05294,문화관광,47.124576,이마트 버스정류소 옆
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.05294,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆
161,61,서울숲방문자센터,서울 성동구 뚝섬로 273,37.539411,127.05294,휴양관광지,47.124576,이마트 버스정류소 옆
184,84,뚝섬두꺼비왕식자재마트,서울 성동구 뚝섬로 4,37.539411,127.05294,쇼핑,47.124576,이마트 버스정류소 옆


In [60]:
place_df = place_df.drop(index=[100, 106, 161, 184])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
108,8,엔터식스/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
116,16,센트라스상가/L동,서울 성동구 왕십리로 41,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
117,17,비트플렉스,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
120,20,옥수리버젠상가,서울 성동구 독서당로 223,37.554347,127.034047,쇼핑,64.296324,응봉삼거리


In [61]:
place_df[place_df["위도"] == place_df.loc[108, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
102,2,이마트/왕십리점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
108,8,엔터식스/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
117,17,비트플렉스,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
126,26,CGV/왕십리,서울 성동구 왕십리광장로 17,37.561222,127.037136,문화관광,40.033302,왕십리역 4번 출구 건너편
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
152,52,엔터식스왕십리역사점1층헐리우드,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
167,67,돔스크린골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
193,93,영풍문고/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편


In [62]:
place_df = place_df.drop(index=[102, 117, 126, 152, 167, 193])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
116,16,센트라스상가/L동,서울 성동구 왕십리로 41,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
120,20,옥수리버젠상가,서울 성동구 독서당로 223,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
121,21,이케아랩/성수점,서울 성동구 아차산로17길 48,37.545982,127.065913,쇼핑,189.595426,화양 APT(횡단보도 옆)
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리


In [63]:
place_df[place_df["위도"] == place_df.loc[116, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
103,3,성동구민종합체육센터,서울 성동구 왕십리로 89,37.549734,127.044168,레포츠(육상/해상/항공),144.288271,서울숲 IT캐슬
116,16,센트라스상가/L동,서울 성동구 왕십리로 41,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
132,32,파크에비뉴엔터식스/한양대점,서울 성동구 왕십리로 241,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
146,46,소월아트홀,서울 성동구 왕십리로 281,37.549734,127.044168,문화관광,144.288271,서울숲 IT캐슬
150,50,다이소/상왕십리역점,서울 성동구 왕십리로 373,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
151,51,센트라스상가/I동,서울특별시 성동구 왕십리로 410,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
183,83,GS25/성동체육센터점,서울 성동구 왕십리로 89,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
189,89,LG전자베스트샵/왕십리점,서울 성동구 왕십리로 314,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬
190,90,소월아트홀,서울 성동구 왕십리로 281,37.549734,127.044168,문화관광,144.288271,서울숲 IT캐슬
194,94,노브랜드/엔터식스한양대점,서울 성동구 왕십리로 241,37.549734,127.044168,쇼핑,144.288271,서울숲 IT캐슬


In [64]:
place_df = place_df.drop(index=[103, 116, 132, 150, 151, 183, 189, 190, 194])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
120,20,옥수리버젠상가,서울 성동구 독서당로 223,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
121,21,이케아랩/성수점,서울 성동구 아차산로17길 48,37.545982,127.065913,쇼핑,189.595426,화양 APT(횡단보도 옆)
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리


In [65]:
place_df[place_df["위도"] == place_df.loc[120, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
109,9,금남시장,서울 성동구 독서당로 303-7,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
120,20,옥수리버젠상가,서울 성동구 독서당로 223,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
188,88,투엑스휘트니스/옥수점,서울 성동구 독서당로 166,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리


In [66]:
place_df = place_df.drop(index=[120, 188])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
121,21,이케아랩/성수점,서울 성동구 아차산로17길 48,37.545982,127.065913,쇼핑,189.595426,화양 APT(횡단보도 옆)
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편


In [67]:
place_df[place_df["위도"] == place_df.loc[121, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
119,19,성수낙낙,서울 성동구 아차산로17길 48,37.545982,127.065913,쇼핑,189.595426,화양 APT(횡단보도 옆)
121,21,이케아랩/성수점,서울 성동구 아차산로17길 48,37.545982,127.065913,쇼핑,189.595426,화양 APT(횡단보도 옆)


In [68]:
place_df = place_df.drop(index=[121])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
141,41,LCDC SEOUL,서울 성동구 연무장17길 1,37.542257,127.061605,문화관광,191.284581,성수사거리 버스정류장 앞


In [69]:
place_df[place_df["위도"] == place_df.loc[131, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
109,9,금남시장,서울 성동구 독서당로 303-7,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리


In [70]:
place_df[place_df["위도"] == place_df.loc[137, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
109,9,금남시장,서울 성동구 독서당로 303-7,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리


In [71]:
place_df[place_df["위도"] == place_df.loc[140, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
108,8,엔터식스/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편


In [72]:
place_df[place_df["위도"] == place_df.loc[141, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
112,12,LCDC SEOUL,서울 성동구 연무장17길 1,37.542257,127.061605,문화관광,191.284581,성수사거리 버스정류장 앞
141,41,LCDC SEOUL,서울 성동구 연무장17길 1,37.542257,127.061605,문화관광,191.284581,성수사거리 버스정류장 앞


In [73]:
place_df = place_df.drop(index=[141])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
143,43,삼성디지털프라자/왕십리점,서울 성동구 고산자로 217,37.555528,127.034289,쇼핑,187.735738,응봉삼거리


In [74]:
place_df[place_df["위도"] == place_df.loc[143, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
128,28,삼성전자서비스/왕십리휴대폰센터,서울 성동구 고산자로 217,37.555528,127.034289,쇼핑,187.735738,응봉삼거리
143,43,삼성디지털프라자/왕십리점,서울 성동구 고산자로 217,37.555528,127.034289,쇼핑,187.735738,응봉삼거리
169,69,다이소/왕십리점,서울 성동구 고산자로 234,37.555528,127.034289,쇼핑,187.735738,응봉삼거리


In [75]:
place_df = place_df.drop(index=[128, 143, 169])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
148,48,그라운드시소/성수,서울 성동구 아차산로17길 49,37.546406,127.065243,문화관광,198.419972,푸조비즈타워 앞


In [76]:
place_df[place_df["위도"] == place_df.loc[148, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
113,13,그라운드시소/성수,서울 성동구 아차산로17길 49,37.546406,127.065243,문화관광,198.419972,푸조비즈타워 앞
148,48,그라운드시소/성수,서울 성동구 아차산로17길 49,37.546406,127.065243,문화관광,198.419972,푸조비즈타워 앞
177,77,그라운드시소/성수,서울 성동구 아차산로17길 49,37.546406,127.065243,문화관광,198.419972,푸조비즈타워 앞


In [77]:
place_df = place_df.drop(index=[113, 148])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
149,49,뚝도시장,서울 성동구 성수이로 32-15,37.538272,127.054630,쇼핑,188.084623,성수2가1동 공영주차장 인근


In [78]:
place_df[place_df["위도"] == place_df.loc[149, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
122,22,라성식자재마트,서울 성동구 성수이로 118,37.538272,127.05463,쇼핑,188.084623,성수2가1동 공영주차장 인근
149,49,뚝도시장,서울 성동구 성수이로 32-15,37.538272,127.05463,쇼핑,188.084623,성수2가1동 공영주차장 인근
179,79,프렌즈아카데미/성수점,서울 성동구 성수이로 147,37.538272,127.05463,레포츠(육상/해상/항공),188.084623,성수2가1동 공영주차장 인근


In [79]:
place_df = place_df.drop(index=[122])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
155,55,서울새활용플라자,서울 성동구 자동차시장길 49,37.560040,127.056370,문화관광,27.095594,성동도로사업소


In [80]:
place_df[place_df["위도"] == place_df.loc[155, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
111,11,XGOLF골프연습장/장한평점,서울 성동구 자동차시장길 42,37.56004,127.05637,레포츠(육상/해상/항공),27.095594,성동도로사업소
155,55,서울새활용플라자,서울 성동구 자동차시장길 49,37.56004,127.05637,문화관광,27.095594,성동도로사업소
163,63,코오롱스포렉스/용답점,서울 성동구 자동차시장길 42,37.56004,127.05637,레포츠(육상/해상/항공),27.095594,성동도로사업소


In [81]:
place_df = place_df.drop(index=[111])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [82]:
place_df[place_df["위도"] == place_df.loc[156, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.05294,휴양관광지,47.124576,이마트 버스정류소 옆
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.05294,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [83]:
place_df[place_df["위도"] == place_df.loc[162, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
144,44,디뮤지엄,서울 성동구 왕십리로 83-21,37.544136,127.044151,문화관광,70.762016,서울숲역 4번 출구 옆
162,62,디뮤지엄,서울 성동구 왕십리로 83-21,37.544136,127.044151,문화관광,70.762016,서울숲역 4번 출구 옆


In [84]:
place_df = place_df.drop(index=[144])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [85]:
place_df[place_df["위도"] == place_df.loc[163, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
155,55,서울새활용플라자,서울 성동구 자동차시장길 49,37.56004,127.05637,문화관광,27.095594,성동도로사업소
163,63,코오롱스포렉스/용답점,서울 성동구 자동차시장길 42,37.56004,127.05637,레포츠(육상/해상/항공),27.095594,성동도로사업소


In [86]:
place_df[place_df["위도"] == place_df.loc[165, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
115,15,피치스도원,서울 성동구 연무장3길 9,37.544928,127.051797,문화관광,107.974467,유니베라 앞
165,65,피치스도원,서울 성동구 연무장3길 9,37.544928,127.051797,문화관광,107.974467,유니베라 앞


In [87]:
place_df = place_df.drop(index=[115])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [88]:
place_df[place_df["위도"] == place_df.loc[166, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
157,57,에디토리,서울 성동구 아차산로 15-8,37.543631,127.058992,쇼핑,86.753571,우영테크노센터 앞
166,66,뚝섬미술관,서울 성동구 아차산로 33,37.543631,127.058992,문화관광,86.753571,우영테크노센터 앞


In [89]:
place_df[place_df["위도"] == place_df.loc[170, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
118,18,응봉산암벽공원,서울 성동구 독서당로60길 13-1,37.549386,127.031115,휴양관광지,98.735343,응봉현대아파트 정류소
170,70,응봉산암벽공원,서울 성동구 독서당로60길 13-1,37.549386,127.031115,문화관광,98.735343,응봉현대아파트 정류소


In [90]:
place_df = place_df.drop(index=[118])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [91]:
place_df[place_df["위도"] == place_df.loc[172, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
105,5,롯데마트/행당역점,서울 성동구 행당로 84,37.555654,127.024965,쇼핑,98.076587,논골사거리(금호도서관 입구)
172,72,대림리빙프라자,서울 성동구 행당로 87,37.555654,127.024965,쇼핑,98.076587,논골사거리(금호도서관 입구)


In [92]:
place_df = place_df.drop(index=[105])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [93]:
place_df[place_df["위도"] == place_df.loc[175, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
125,25,에스팩토리,서울 성동구 연무장15길 11,37.542062,127.059868,문화관광,141.042752,우영테크노센터 앞
175,75,에스팩토리,서울 성동구 연무장15길 11,37.542062,127.059868,문화관광,141.042752,우영테크노센터 앞


In [94]:
place_df = place_df.drop(index=[125])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [95]:
place_df[place_df["위도"] == place_df.loc[179, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
149,49,뚝도시장,서울 성동구 성수이로 32-15,37.538272,127.05463,쇼핑,188.084623,성수2가1동 공영주차장 인근
179,79,프렌즈아카데미/성수점,서울 성동구 성수이로 147,37.538272,127.05463,레포츠(육상/해상/항공),188.084623,성수2가1동 공영주차장 인근


In [96]:
place_df[place_df["위도"] == place_df.loc[180, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
133,33,레몬프라자,서울 성동구 고산자로6길 4,37.557844,127.034963,쇼핑,109.227988,kt 행당빌딩 앞
180,80,롯데슈퍼/행당점,서울 성동구 고산자로6길 4,37.557844,127.034963,쇼핑,109.227988,kt 행당빌딩 앞


In [97]:
place_df = place_df.drop(index=[180])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [98]:
place_df[place_df["위도"] == place_df.loc[187, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
176,76,아인테니스클럽,서울 성동구 광나루로 286,37.551567,127.045875,레포츠(육상/해상/항공),56.904636,더스페이스타워 앞
187,87,에딘버그골프클럽,서울 성동구 광나루로 320-1,37.551567,127.045875,레포츠(육상/해상/항공),56.904636,더스페이스타워 앞


In [99]:
place_df = place_df.drop(index=[187])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [100]:
place_df[place_df["위도"] == place_df.loc[191, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
129,29,롯데하이마트/장한평점,서울 성동구 천호대로 394,37.56724,127.052108,쇼핑,114.878817,답십리역 8번출구 앞
191,91,다이소/답십리점,서울 성동구 천호대로 308,37.56724,127.052108,쇼핑,114.878817,답십리역 8번출구 앞


In [101]:
place_df = place_df.drop(index=[129, 191])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [102]:
place_df[place_df["위도"] == place_df.loc[205, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
198,2,경복궁,서울 종로구 사직로 161,37.575918,126.976829,역사관광지,191.085618,광화문 시민열린마당
205,9,경복궁,서울 종로구 사직로 161,37.575918,126.976829,역사관광지,191.085618,광화문 시민열린마당
213,17,광화문,서울 종로구 사직로 161,37.575918,126.976829,역사관광지,191.085618,광화문 시민열린마당
252,56,광화문,서울 종로구 사직로 161,37.575918,126.976829,역사관광지,191.085618,광화문 시민열린마당


In [103]:
place_df.loc[252, "관광지명"] = "경복궁+광화문"
place_df = place_df.drop(index=[198, 205, 213])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [104]:
place_df[place_df["위도"] == place_df.loc[219, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
204,8,낙산공원,서울 종로구 낙산길 41,37.581405,127.013138,휴양관광지,51.800594,쌍용아파트2단지 정문
219,23,낙산공원,서울 종로구 낙산길 41,37.581405,127.013138,문화관광,51.800594,쌍용아파트2단지 정문


In [105]:
place_df = place_df.drop(index=[204])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [106]:
place_df[place_df["위도"] == place_df.loc[234, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
210,14,북촌한옥마을,서울특별시 종로구 계동길 37,37.579046,126.986388,역사관광지,164.16207,재동초교 앞 삼거리
234,38,북촌한옥마을,서울특별시 종로구 계동길 37,37.579046,126.986388,역사관광지,164.16207,재동초교 앞 삼거리


In [107]:
place_df = place_df.drop(index=[210])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [108]:
place_df[place_df["위도"] == place_df.loc[247, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
216,20,서울역사박물관,서울 종로구 새문안로 55,37.570478,126.970526,문화관광,82.859018,서울역사박물관 앞
247,51,서울역사박물관,서울 종로구 새문안로 55,37.570478,126.970526,문화관광,82.859018,서울역사박물관 앞


In [109]:
place_df = place_df.drop(index=[216])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [110]:
place_df[place_df["위도"] == place_df.loc[261, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
225,29,청와대사랑채,서울 종로구 효자로13길 45,37.582493,126.973094,문화관광,137.176169,효자동 삼거리
261,65,청와대사랑채,서울 종로구 효자로13길 45,37.582493,126.973094,문화관광,137.176169,효자동 삼거리


In [111]:
place_df = place_df.drop(index=[225])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [112]:
# 생각보다 관광지명이 아예 똑같은 것들이 너무 많아서...
place_df = place_df.drop_duplicates(["관광지명"], keep="first")

In [113]:
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [114]:
place_df[place_df["위도"] == place_df.loc[263, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
201,5,동대문종합시장,서울 종로구 종로 266,37.570382,127.007772,쇼핑,121.281697,종로꽃시장 입구 옆
263,67,동대문쇼핑타운,서울 종로구 종로 266,37.570382,127.007772,쇼핑,121.281697,종로꽃시장 입구 옆
271,75,동대문종합시장/A동,서울 종로구 종로 266,37.570382,127.007772,쇼핑,121.281697,종로꽃시장 입구 옆
277,81,동대문종합시장악세사리상가,서울 종로구 종로 266,37.570382,127.007772,쇼핑,121.281697,종로꽃시장 입구 옆
281,85,동대문종합시장/D동,서울 종로구 종로 266,37.570382,127.007772,쇼핑,121.281697,종로꽃시장 입구 옆


In [115]:
place_df = place_df.drop(index=[201, 271, 277, 281])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [116]:
place_df[place_df["위도"] == place_df.loc[265, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
264,68,탑골공원,서울 종로구 종로 99,37.57157,126.988186,휴양관광지,130.229102,탑골공원 앞
265,69,인왕산,서울 종로구 종로 99,37.57157,126.988186,휴양관광지,130.229102,탑골공원 앞


In [117]:
place_df = place_df.drop(index=[265])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [118]:
place_df[place_df["위도"] == place_df.loc[331, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
301,5,롯데백화점/강남점,서울 강남구 도곡로 401,37.496969,127.053241,쇼핑,38.662964,한티역 롯데백화점 앞
331,35,더콘란샵,서울 강남구 도곡로 401,37.496969,127.053241,쇼핑,38.662964,한티역 롯데백화점 앞


In [119]:
place_df = place_df.drop(index=[331])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [120]:
place_df[place_df["위도"] == place_df.loc[340, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
297,1,현대백화점/무역센터점,서울 강남구 테헤란로 517,37.508608,127.059745,쇼핑,114.104367,섬유센터 앞
340,44,현대백화점면세점/무역센터점,서울 강남구 테헤란로 517,37.508608,127.059745,쇼핑,114.104367,섬유센터 앞


In [121]:
place_df = place_df.drop(index=[297])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [122]:
place_df[place_df["위도"] == place_df.loc[342, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
305,9,신사동가로수길,서울 강남구,37.5177,127.0473,휴양관광지,45.643967,강남구청
342,46,압구정로데오거리,서울 강남구,37.5177,127.0473,휴양관광지,45.643967,강남구청


In [123]:
place_df.loc[305, "관광지명"] = "신사동가로수길+압구정로데오"
place_df = place_df.drop(index=[342])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [124]:
place_df[place_df["위도"] == place_df.loc[347, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
344,48,하나로마트/대치점,서울 강남구 삼성로 15,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)
347,51,해리슨테일러/청담본점,서울 강남구 삼성로 719,37.506723,127.056661,쇼핑,60.385559,포스코사거리(기업은행)


In [125]:
place_df = place_df.drop(index=[344])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [126]:
place_df[place_df["위도"] == place_df.loc[359, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
319,23,삼성디지털프라자/메가스토어삼성대치,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆
359,63,삼성전자서비스/대치휴대폰센터,서울 강남구 영동대로 237,37.500604,127.067025,쇼핑,89.707696,천주교 대치 2동 교회 옆


In [127]:
place_df = place_df.drop(index=[319, 359])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [128]:
place_df[place_df["위도"] == place_df.loc[381, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
328,32,SETEC,서울 강남구 남부순환로 3104,37.49348,127.061274,문화관광,191.756747,대치역 7번출구
381,85,다이소/매봉역점,서울 강남구 남부순환로 2748,37.49348,127.061274,쇼핑,191.756747,대치역 7번출구


In [129]:
place_df = place_df.drop(index=[381])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [130]:
place_df[place_df["위도"] == place_df.loc[395, "위도"]]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
369,73,반가의우리옷/청담본점,서울 강남구 삼성로126길 6,37.517741,127.05179,쇼핑,107.88865,강남구 도시관리공단
395,99,엘레바또,서울 강남구 삼성로126길 6,37.517741,127.05179,쇼핑,107.88865,강남구 도시관리공단


In [131]:
place_df = place_df.drop(index=[369])
place_df[place_df.duplicated(["위도", "경도"])].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
74,74,숭례문수입상가,서울 중구 소월로 3,37.555870,126.976870,쇼핑,68.640827,도동삼거리
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆


In [132]:
# 필요없는/중복 자료 없애고 나니 약 150개 가량만 남은 것 확인 가능...
place_df

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
0,0,롯데백화점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
9,9,남대문시장,서울 중구,37.556028,126.972224,쇼핑,125.327570,서울역9번출구
10,10,롯데마트/서울역점,서울 중구 청파로 426,37.558630,126.969327,쇼핑,156.613953,종로학원본원
13,13,롯데영플라자/명동점,서울 중구 남대문로 67,37.568713,126.982673,쇼핑,40.548823,광교사거리 남측
...,...,...,...,...,...,...,...,...
389,93,에스엘알렌트/강남점,서울 강남구 테헤란로6길 9,37.497111,127.031020,쇼핑,180.583230,역삼.서초.삼성 세무서 앞 (역삼빌딩 앞)
390,94,숙현한복,서울 강남구 학동로55길 12-6,37.518832,127.042086,쇼핑,156.241944,학동로 래미안 아파트 앞
393,97,두오모앤코,서울 강남구 논현로132길 25,37.515386,127.032697,쇼핑,103.129390,서울 논현동우체국 앞
394,98,한복이야기아씨,서울 강남구 도산대로56길 1,37.522533,127.040267,쇼핑,120.284140,청담동 맥도날드 옆(위치)


In [133]:
place_df["분류"].value_counts()

쇼핑               81
문화관광             36
레포츠(육상/해상/항공)    15
휴양관광지             9
역사관광지             7
Name: 분류, dtype: int64

In [136]:
# 각 항목 별로 한 번씩만 체크해보기
place_df[place_df["분류"] == "쇼핑"].head(20)

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
0,0,롯데백화점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
9,9,남대문시장,서울 중구,37.556028,126.972224,쇼핑,125.327570,서울역9번출구
10,10,롯데마트/서울역점,서울 중구 청파로 426,37.558630,126.969327,쇼핑,156.613953,종로학원본원
13,13,롯데영플라자/명동점,서울 중구 남대문로 67,37.568713,126.982673,쇼핑,40.548823,광교사거리 남측
16,16,디오트쇼핑몰,서울 중구 다산로 293,37.567665,127.016085,쇼핑,77.372160,신당역 10번 출구
17,17,누죤,서울 중구 을지로45길 62,37.566900,127.013010,쇼핑,156.727888,ddp패션몰
19,19,ddp패션몰,서울 중구 마장로 22,37.568005,127.015818,쇼핑,108.075606,신당역 10번 출구
27,27,굿모닝시티,서울 중구 장충단로 247,37.566375,127.007386,쇼핑,122.595092,동대문역사문화공원역 1번출구 뒤편
28,28,HelloAPM,서울 중구 장충단로 253,37.567429,127.008059,쇼핑,134.252690,동대문역사문화공원역 1번출구 뒤편


In [137]:
place_df = place_df.drop(index=[70])
place_df[place_df["분류"] == "쇼핑"][20:]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
85,85,디자이너클럽,서울 중구 을지로45길 72,37.567591,127.012600,쇼핑,87.636003,ddp패션몰
86,86,롯데하이마트/서울역롯데마트점,서울 중구 한강대로 405,37.554555,126.970779,쇼핑,177.447173,서울역 서부교차로(3)
88,88,청계천 쇼핑거리,서울 중구 청계천로 16,37.571681,127.021172,쇼핑,96.519781,황학동롯데캐슬
89,89,메사,서울 중구 남대문시장10길 2,37.560381,126.980050,쇼핑,70.719281,회현역 7번출구
91,91,남평화상가 + 밀리오레/동대문점,서울 중구 장충단로 282-10,37.568043,127.009194,쇼핑,42.114914,동대문디자인플라자
99,99,아영에프비씨,서울 중구 퇴계로46길 23,37.561778,126.999843,쇼핑,188.430414,퇴계로4가 교차로(중구청)
108,8,엔터식스/왕십리역점,서울 성동구 왕십리광장로 17,37.561222,127.037136,쇼핑,40.033302,왕십리역 4번 출구 건너편
109,9,금남시장,서울 성동구 독서당로 303-7,37.554347,127.034047,쇼핑,64.296324,응봉삼거리
114,14,텐즈힐몰,서울 성동구 마장로 137,37.567463,127.044969,쇼핑,175.393261,마장동 주민센터
119,19,성수낙낙,서울 성동구 아차산로17길 48,37.545982,127.065913,쇼핑,189.595426,화양 APT(횡단보도 옆)


In [138]:
place_df = place_df.drop(index=[86, 134, 158, 181, 203, 218, 231, 262, 318, 346, 348])

In [139]:
place_df[place_df["분류"] == "문화관광"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
44,44,안중근의사기념관,서울 중구 소월로 91,37.555870,126.976870,문화관광,68.640827,도동삼거리
51,51,국립현대미술관/덕수궁관,서울 중구 세종대로 99,37.566331,126.973900,문화관광,152.248120,시립미술관
53,53,서울시립미술관,서울 중구 덕수궁길 61,37.564160,126.973808,문화관광,108.646323,시립미술관
61,61,국립극장,서울 중구 장충단로 59,37.568043,127.009194,문화관광,42.114914,동대문디자인플라자
67,67,피크닉,서울 중구 퇴계로6가길 3,37.556244,126.978004,문화관광,64.922452,도동삼거리
68,68,화폐박물관,서울 중구 남대문로 39,37.568713,126.982673,문화관광,40.548823,광교사거리 남측
69,69,서울애니메이션센터,서울 중구 소공로 48,37.559784,126.982936,문화관광,71.262743,회현사거리 남측
78,78,충무아트센터,서울 중구 퇴계로 387,37.565690,127.014736,문화관광,185.387691,신당역 12번 출구 뒤
80,80,국립정동극장,서울 중구 정동길 43,37.565865,126.972772,문화관광,114.114119,시립미술관
94,94,조선일보미술관,서울 중구 세종대로21길 33,37.567614,126.975763,문화관광,136.191140,서울도시건축전시관


In [140]:
place_df[place_df["분류"] == "레포츠(육상/해상/항공)"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
73,73,장충장호테니스장,서울 중구 장충단로 91,37.568043,127.009194,레포츠(육상/해상/항공),42.114914,동대문디자인플라자
87,87,QED골프아카데미/충정로직영4호점,서울 중구 서소문로 38,37.562204,126.970061,레포츠(육상/해상/항공),73.478129,호암아트홀
130,30,디엣지아이스링크,서울 성동구 자동차시장1길 18,37.562045,127.057751,레포츠(육상/해상/항공),135.288835,하나은행 장안동지점앞
137,37,더프라자 골프/옥수점,서울 성동구 독서당로 202,37.554347,127.034047,레포츠(육상/해상/항공),64.296324,응봉삼거리
140,40,돔골프,서울 성동구 왕십리광장로 17,37.561222,127.037136,레포츠(육상/해상/항공),40.033302,왕십리역 4번 출구 건너편
153,53,금호스포츠센터,서울 성동구 금호로 3,37.550240,127.026028,레포츠(육상/해상/항공),181.425217,금호사거리
156,56,희소볼링장,서울 성동구 뚝섬로 447,37.539411,127.052940,레포츠(육상/해상/항공),47.124576,이마트 버스정류소 옆
163,63,코오롱스포렉스/용답점,서울 성동구 자동차시장길 42,37.560040,127.056370,레포츠(육상/해상/항공),27.095594,성동도로사업소
171,71,성동구립용답체육센터,서울 성동구 천호대로78길 15-48,37.560984,127.055115,레포츠(육상/해상/항공),187.819976,성동도로사업소
176,76,아인테니스클럽,서울 성동구 광나루로 286,37.551567,127.045875,레포츠(육상/해상/항공),56.904636,더스페이스타워 앞


In [141]:
place_df = place_df.drop(index=[87, 295])
place_df[place_df["분류"] == "휴양관광지"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
14,14,남산공원,서울 중구 소파로 46,37.556140,126.979130,휴양관광지,188.458164,도동삼거리
71,71,만리재길+남산백범광장,서울 중구,37.556028,126.972224,휴양관광지,125.327570,서울역9번출구
101,1,서울숲,서울 성동구 뚝섬로 273,37.539411,127.052940,휴양관광지,47.124576,이마트 버스정류소 옆
229,33,청와대앞길,서울 종로구 청와대로 1,37.582369,126.980198,휴양관광지,140.146442,청와대앞길
236,40,대학로,서울 종로구 대학로 12,37.573654,127.002052,휴양관광지,153.621165,서울보증보험본사 앞
246,50,마로니에공원,서울 종로구 대학로 104,37.582080,127.001969,휴양관광지,60.229563,혜화역 3번출구 뒤
264,68,탑골공원,서울 종로구 종로 99,37.571570,126.988186,휴양관광지,130.229102,탑골공원 앞
282,86,사직공원,서울 종로구 사직로 89,37.576299,126.968583,휴양관광지,3.716450,사직동주민센터
305,9,신사동가로수길+압구정로데오,서울 강남구,37.517700,127.047300,휴양관광지,45.643967,강남구청


In [143]:
place_df.loc[236, "관광지명"] = "대학로/마로니에공원"
place_df = place_df.drop(index=[246])
place_df[place_df["분류"] == "역사관광지"]

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
21,21,덕수궁,서울 중구 세종대로 99,37.566331,126.973900,역사관광지,152.248120,시립미술관
56,56,숭례문,서울 중구 세종대로 4,37.566864,126.977367,역사관광지,61.054778,서울도시건축전시관
63,63,서소문성지역사박물관,서울 중구 칠패로 5,37.559443,126.968909,역사관광지,58.204214,종로학원본원
131,31,응봉산팔각정,서울특별시 성동구 독서당로 362,37.554347,127.034047,역사관광지,64.296324,응봉삼거리
234,38,북촌한옥마을,서울특별시 종로구 계동길 37,37.579046,126.986388,역사관광지,164.162070,재동초교 앞 삼거리
252,56,경복궁+광화문,서울 종로구 사직로 161,37.575918,126.976829,역사관광지,191.085618,광화문 시민열린마당
285,89,흥인지문,서울 종로구 종로 288,37.571111,127.009607,역사관광지,110.665673,동대문역 8번 출구


In [144]:
place_df

,순위,관광지명,주소,위도,경도,분류,최단거리,최단거리대여소
0,0,롯데백화점/본점,서울 중구 남대문로 81,37.564856,126.981707,쇼핑,116.515324,을지로입구역 8번출구
2,2,신세계백화점/본점,서울 중구 소공로 63,37.560294,126.980503,쇼핑,55.278224,신세계면세점
9,9,남대문시장,서울 중구,37.556028,126.972224,쇼핑,125.327570,서울역9번출구
10,10,롯데마트/서울역점,서울 중구 청파로 426,37.558630,126.969327,쇼핑,156.613953,종로학원본원
13,13,롯데영플라자/명동점,서울 중구 남대문로 67,37.568713,126.982673,쇼핑,40.548823,광교사거리 남측
...,...,...,...,...,...,...,...,...
389,93,에스엘알렌트/강남점,서울 강남구 테헤란로6길 9,37.497111,127.031020,쇼핑,180.583230,역삼.서초.삼성 세무서 앞 (역삼빌딩 앞)
390,94,숙현한복,서울 강남구 학동로55길 12-6,37.518832,127.042086,쇼핑,156.241944,학동로 래미안 아파트 앞
393,97,두오모앤코,서울 강남구 논현로132길 25,37.515386,127.032697,쇼핑,103.129390,서울 논현동우체국 앞
394,98,한복이야기아씨,서울 강남구 도산대로56길 1,37.522533,127.040267,쇼핑,120.284140,청담동 맥도날드 옆(위치)


In [145]:
place_df.to_csv("관광지_전처리끝.csv")